# 0. Premilinary Works
How big is the file, number of rows, etc.

In [3]:
cd ../data

/home/angelrps/git/MasterDataScience_FinalProject/data


In [4]:
!ls -Rl
# 2017_Yellow_Taxi_Trip_Data.csv is 10532305815 bytes (9,80 GB)
# 2018_Yellow_Taxi_Trip_Data.csv is 10428263736 bytes (9,71 GB)
# 2019_Yellow_Taxi_Trip_Data.csv is 8197837930 bytes (7,63 GB)

.:
total 2897760
-rwxrwxrwx 1 angelrps angelrps 1048588401 Mar 25 13:15 2017_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps 1065918551 Jun 18 12:58 2018_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps  835929368 Apr  2 18:02 2019_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps     564780 Jun 18 11:15 Data_Weather_Cleaned.csv
-rwxrwxrwx 1 angelrps angelrps     436485 Jun 16 10:51 LCD_documentation.pdf
-rwxrwxrwx 1 angelrps angelrps   13843633 Jun 16 09:40 NOAA_CentralPark_Weather.csv
-rwxrwxrwx 1 angelrps angelrps     196848 Apr  7 19:34 data_dictionary_trip_records_yellow.pdf
-rwxrwxrwx 1 angelrps angelrps    1489310 Apr  7 19:34 taxi_zone_map_manhattan.jpg
-rwxrwxrwx 1 angelrps angelrps     202694 Apr  4 12:46 trip_record_user_guide.pdf


In [4]:
# I am dealing with very heavy files so I will compress them (bz2) to work with them
# and remove the heavy .csv from my drive
!bzip2 2017_Yellow_Taxi_Trip_Data.csv 2018_Yellow_Taxi_Trip_Data.csv 2019_Yellow_Taxi_Trip_Data.csv

In [5]:
!ls -Rl

.:
total 2897760
-rwxrwxrwx 1 angelrps angelrps 1048588401 Mar 25 13:15 2017_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps 1065918551 Jun 18 12:58 2018_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps  835929368 Apr  2 18:02 2019_Yellow_Taxi_Trip_Data.csv.bz2
-rwxrwxrwx 1 angelrps angelrps     564780 Jun 18 11:15 Data_Weather_Cleaned.csv
-rwxrwxrwx 1 angelrps angelrps     436485 Jun 16 10:51 LCD_documentation.pdf
-rwxrwxrwx 1 angelrps angelrps   13843633 Jun 16 09:40 NOAA_CentralPark_Weather.csv
-rwxrwxrwx 1 angelrps angelrps     196848 Apr  7 19:34 data_dictionary_trip_records_yellow.pdf
-rwxrwxrwx 1 angelrps angelrps    1489310 Apr  7 19:34 taxi_zone_map_manhattan.jpg
-rwxrwxrwx 1 angelrps angelrps     202694 Apr  4 12:46 trip_record_user_guide.pdf


In [8]:
# Count Number of Lines
!zgrep -c $ 2017_Yellow_Taxi_Trip_Data.csv.bz2
!zgrep -c $ 2018_Yellow_Taxi_Trip_Data.csv.bz2
!zgrep -c $ 2019_Yellow_Taxi_Trip_Data.csv.bz2

# 2017_...10.748.623 lines
# 2018_...10.734.230 lines
# 2019_...8.402.723 lines

10748623
10734230
8402723


#### I will start exploring the 2019 dataset
I will try to use bash to clean the data as much as I can before importing a Data Frame because I believe is faster.
From a previous exploration I concluded:
1. There are trips with 0 'trip_distance'. I will remove all trips with 'trip_distance' < 0.06 (100 meters) because I consider them either measurement errors or non representative data.
2. There are trips with either negative or very little 'fare_amount' (~0.01). I will remove rows with fare amount<1$ because I consider them measurement errors.

In [11]:
!bzcat ./2019_Yellow_Taxi_Trip_Data.csv.bz2 | head -n 5

# Conclusions:
# 1. There is a header on the first line
# 2. The separator is ','

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
1,04/29/2019 11:31:03 AM,04/29/2019 11:31:03 AM,1,0,1,N,231,264,2,2.5,2.5,0.5,0,0,0.3,5.8,2.5
2,04/29/2019 10:52:08 AM,04/29/2019 11:06:18 AM,1,1.01,1,N,186,230,1,10,0,0.5,2.66,0,0.3,15.96,2.5
2,04/29/2019 11:28:40 AM,04/29/2019 11:33:01 AM,1,0.82,1,N,238,151,1,5,0,0.5,1.66,0,0.3,9.96,2.5
1,04/29/2019 11:28:06 AM,04/29/2019 12:07:32 PM,1,12.1,1,N,138,88,1,38.5,2.5,0.5,10.4,0,0.3,52.2,2.5

bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = ./2019_Yellow_Taxi_Trip_Data.csv.bz2, output file = (stdout)


### 1. Clean up dataset taking just a sample

The following cleaning actions are based on the dataset exploration made in the notebok 'Data_Analysis.ipynb'

In [1]:
# LOAD LIBRARIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
pd.options.display.max_columns = None
pd.options.display.max_rows = None

Import part of the csv so the memory can handle it

In [35]:
# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
#    Import just needed columns to lighten the process
dfsample = pd.read_csv('./2019_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})
dfsample.shape

(9999, 4)

Let's remove trips under 0.06 (100m). I consider them either errors or not relevant

In [34]:
# RECAP 2

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
#    Import just needed columns to lighten the process
dfsample = pd.read_csv('./2019_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Remove all trips with 'trip_distance' < 0.06 (100m) because I consider them measurement errors or non representative values
DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)
dfsample[dfsample['trip_distance']<0.06].index # this should be none

Int64Index([], dtype='int64')

Now let's remove negative 'fare_amount' or <1$. I consider them errors or not relevant.

In [28]:
# RECAP 3

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
#    Import just needed columns to lighten the process
dfsample = pd.read_csv('./2019_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Remove all rows with 'trip_distance' < 0.06 (100m) because I consider them measurement errors or non representative values
DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)

# 3. Remove rows with 'fare_amount' negative or <1$ because I consider them measurement errors or non relevant.
FareIndex_to_drop = dfsample[dfsample['fare_amount']<1].index
dfsample.drop(FareIndex_to_drop, inplace=True)
dfsample[dfsample['fare_amount']<1] # this should be none

,pickup_datetime,VendorID,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge


Once we have removed unwanted rows, let´s select the two columns of interest:
- 'pickup_datetime'
- 'PULocationID'

In [30]:
# RECAP 4

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
#    Import just needed columns to lighten the process
dfsample = pd.read_csv('./2019_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Remove all rows with 'trip_distance' < 0.06 (100m) because I consider them measurement errors or non representative values
DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)

# 3. Remove rows with 'fare_amount' negative or <1$ because I consider them measurement errors or non relevant.
FareIndex_to_drop = dfsample[dfsample['fare_amount']<1].index
dfsample.drop(FareIndex_to_drop, inplace=True)

# 4. Select columns of interest.
#    'pickup_datetime'
#    'PULocationID'
dfsample = dfsample[['pickup_datetime','PULocationID']]
dfsample.head()

,pickup_datetime,PULocationID
1,2019-04-29 10:52:08,186
2,2019-04-29 11:28:40,238
3,2019-04-29 11:28:06,138
4,2019-04-29 11:39:32,138
5,2019-04-29 11:06:45,141


Set timestamp as index

In [33]:
# RECAP 5

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
#    Import just needed columns to lighten the process
dfsample = pd.read_csv('./2019_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Remove all rows with 'trip_distance' < 0.06 (100m) because I consider them measurement errors or non representative values
DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)

# 3. Remove rows with 'fare_amount' negative or <1$ because I consider them measurement errors or non relevant.
FareIndex_to_drop = dfsample[dfsample['fare_amount']<1].index
dfsample.drop(FareIndex_to_drop, inplace=True)

# 4. Select columns of interest.
#    'pickup_datetime'
#    'PULocationID'
dfsample = dfsample[['pickup_datetime','PULocationID']]

# 5. Set timestamp as index
dfsample.set_index('pickup_datetime', inplace=True)
dfsample.head()

,PULocationID
pickup_datetime,
2019-04-29 10:52:08,186
2019-04-29 11:28:40,238
2019-04-29 11:28:06,138
2019-04-29 11:39:32,138
2019-04-29 11:06:45,141


Convert datetime in periods of 1 hour

In [36]:
# RECAP 6

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
#    Import just needed columns to lighten the process
dfsample = pd.read_csv('./2019_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Remove all rows with 'trip_distance' < 0.06 (100m) because I consider them measurement errors or non representative values
DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)

# 3. Remove rows with 'fare_amount' negative or <1$ because I consider them measurement errors or non relevant.
FareIndex_to_drop = dfsample[dfsample['fare_amount']<1].index
dfsample.drop(FareIndex_to_drop, inplace=True)

# 4. Select columns of interest.
#    'pickup_datetime'
#    'PULocationID'
dfsample = dfsample[['pickup_datetime','PULocationID']]

# 5. Set timestamp as index
dfsample.set_index('pickup_datetime', inplace=True)

# 6. Convert the DATE in PERIODS of 1 hour
dfsample = dfsample.to_period("H")
dfsample.head()

,PULocationID
pickup_datetime,
2019-04-29 10:00,186
2019-04-29 11:00,238
2019-04-29 11:00,138
2019-04-29 11:00,138
2019-04-29 11:00,141


Sort by datetime and location

In [38]:
# RECAP 7

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
#    Import just needed columns to lighten the process
dfsample = pd.read_csv('./2019_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Remove all rows with 'trip_distance' < 0.06 (100m) because I consider them measurement errors or non representative values
DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)

# 3. Remove rows with 'fare_amount' negative or <1$ because I consider them measurement errors or non relevant.
FareIndex_to_drop = dfsample[dfsample['fare_amount']<1].index
dfsample.drop(FareIndex_to_drop, inplace=True)

# 4. Select columns of interest.
#    'pickup_datetime'
#    'PULocationID'
dfsample = dfsample[['pickup_datetime','PULocationID']]

# 5. Set timestamp as index
dfsample.set_index('pickup_datetime', inplace=True)

# 6. Convert the DATE in PERIODS of 1 hour
dfsample = dfsample.to_period("H")

# 7. Sort by datetime and by Location
dfsample.sort_values(['pickup_datetime', 'PULocationID'], inplace=True)
dfsample.head(100)

,PULocationID
pickup_datetime,
2019-04-29 05:00,90
2019-04-29 06:00,68
2019-04-29 06:00,88
2019-04-29 06:00,107
2019-04-29 06:00,113
2019-04-29 06:00,125
2019-04-29 06:00,132
2019-04-29 06:00,138
2019-04-29 06:00,236


Create new column to count Number of Pickups per zone

In [20]:
# RECAP 8

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
#    Import just needed columns to lighten the process
dfsample = pd.read_csv('./2019_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Remove all rows with 'trip_distance' < 0.06 (100m) because I consider them measurement errors or non representative values
DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)

# 3. Remove rows with 'fare_amount' negative or <1$ because I consider them measurement errors or non relevant.
FareIndex_to_drop = dfsample[dfsample['fare_amount']<1].index
dfsample.drop(FareIndex_to_drop, inplace=True)

# 4. Select columns of interest.
#    'pickup_datetime'
#    'PULocationID'
dfsample = dfsample[['pickup_datetime','PULocationID']]

# 5. Set timestamp as index
dfsample.set_index('pickup_datetime', inplace=True)

# 6. Convert the DATE in PERIODS of 1 hour
dfsample = dfsample.to_period("H")

# 7. Sort by datetime and by Location
dfsample.sort_values(['pickup_datetime', 'PULocationID'], inplace=True)

# 8. Create new column to count Number of Pickups per zone
dfsample['NoOfPickups'] =1

Groupby Datetime and Location to get Pickups sum()

In [53]:
# RECAP 9

# 1. Prepare sample file with 10.000 rows
#    Parse 'tpep_pickup_datetime' column to datetime object type
#    Import just needed columns to lighten the process
dfsample = pd.read_csv('./2019_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Remove all rows with 'trip_distance' < 0.06 (100m) because I consider them measurement errors or non representative values
DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)

# 3. Remove rows with 'fare_amount' negative or <1$ because I consider them measurement errors or non relevant.
FareIndex_to_drop = dfsample[dfsample['fare_amount']<1].index
dfsample.drop(FareIndex_to_drop, inplace=True)

# 4. Select columns of interest.
#    'pickup_datetime'
#    'PULocationID'
dfsample = dfsample[['pickup_datetime','PULocationID']]

# 5. Set timestamp as index
dfsample.set_index('pickup_datetime', inplace=True)

# 6. Convert the DATE in PERIODS of 1 hour
dfsample = dfsample.to_period("H")

# 7. Sort by datetime and by Location
dfsample.sort_values(['pickup_datetime', 'PULocationID'], inplace=True)

# 8. Create new column to count Number of Pickups per zone
dfsample['NoOfPickups'] =1

# 9. Groupby Datetime and Location to get Pickups sum()
dfsample=dfsample.groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()
dfsample.shape

(279, 3)

### 2. Test Big Data approach on a sample

In [40]:
# LIBRARIES
import pandas as pd
pd.set_option('display.max_columns', None)

In [41]:
# CODE TO RUN

DistIndex_to_drop = dfsample[dfsample['trip_distance'] < 0.06].index
dfsample.drop(DistIndex_to_drop, inplace=True)

FareIndex_to_drop = dfsample[dfsample['fare_amount']<1].index
dfsample.drop(FareIndex_to_drop, inplace=True)

dfsample = dfsample[['pickup_datetime','PULocationID']]

dfsample.set_index('pickup_datetime', inplace=True)

dfsample = dfsample.to_period("H")

dfsample.sort_values(['pickup_datetime', 'PULocationID'], inplace=True)

dfsample['NoOfPickups'] =1

dfsample=dfsample.groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()

In [83]:
# ITERATION CODE WITH CHUNKS

# 1. Get 10000 rows sample iterator with 3000 chunks
dfsi = pd.read_csv('./2019_Yellow_Taxi_Trip_Data.csv.bz2', sep=',',
                       usecols=['tpep_pickup_datetime','PULocationID','trip_distance','fare_amount'],
                       nrows=9999,
                       chunksize=3000,
                       parse_dates={'pickup_datetime':['tpep_pickup_datetime']})

# 2. Declare empty DataFrame to store results of each chunk
all_chunks=pd.DataFrame()

# 3. Iterate through chunks and append results
for i, dfs in enumerate(dfsi):
    print(i, len(dfs))    
    
    # CODE TO RUN
    DistIndex_to_drop = dfs[dfs['trip_distance'] < 0.06].index
    dfs.drop(DistIndex_to_drop, inplace=True)
    
    FareIndex_to_drop = dfs[dfs['fare_amount']<1].index
    dfs.drop(FareIndex_to_drop, inplace=True)

    dfs = dfs[['pickup_datetime','PULocationID']]

    dfs.set_index('pickup_datetime', inplace=True)

    dfs = dfs.to_period("H")

    dfs.sort_values(['pickup_datetime', 'PULocationID'], inplace=True)

    dfs['NoOfPickups'] =1
    
    result=dfs.groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()
    all_chunks=all_chunks.append(result)

#groupby puts the grouping column as index so we have to:
#reset_index()
#groupby again the all_chunks results > sum > sort > reset_index again
result_all = all_chunks.sort_values(['pickup_datetime', 'PULocationID']). \
                                    groupby(['pickup_datetime', 'PULocationID'])['NoOfPickups'].sum().reset_index()
result_all.head()

0 3000
1 3000
2 3000
3 999


,pickup_datetime,PULocationID,NoOfPickups
0,2019-04-29 05:00,90,1
1,2019-04-29 06:00,68,1
2,2019-04-29 06:00,88,1
3,2019-04-29 06:00,107,1
4,2019-04-29 06:00,113,1


I check that the 'result_all' dataframe and 'dfsample' are the same:
- Shape is  in both (279,3)
- Head and Tail rows are the same
- Taking a random row (e.g. row 52), they are both the same.

In [84]:
print(dfsample.tail())
print(result_all.tail())

      pickup_datetime  PULocationID  NoOfPickups
274  2019-04-29 13:00           229            1
275  2019-04-29 13:00           230            1
276  2019-04-29 13:00           231            1
277  2019-04-29 13:00           234            1
278  2019-04-29 13:00           236            2
      pickup_datetime  PULocationID  NoOfPickups
274  2019-04-29 13:00           229            1
275  2019-04-29 13:00           230            1
276  2019-04-29 13:00           231            1
277  2019-04-29 13:00           234            1
278  2019-04-29 13:00           236            2


In [85]:
print(dfsample.head())
print(result_all.head())

    pickup_datetime  PULocationID  NoOfPickups
0  2019-04-29 05:00            90            1
1  2019-04-29 06:00            68            1
2  2019-04-29 06:00            88            1
3  2019-04-29 06:00           107            1
4  2019-04-29 06:00           113            1
    pickup_datetime  PULocationID  NoOfPickups
0  2019-04-29 05:00            90            1
1  2019-04-29 06:00            68            1
2  2019-04-29 06:00            88            1
3  2019-04-29 06:00           107            1
4  2019-04-29 06:00           113            1


In [86]:
print(dfsample.loc[52])
print(result_all.loc[52])

pickup_datetime    2019-04-29 10:00
PULocationID                    237
NoOfPickups                       4
Name: 52, dtype: object
pickup_datetime    2019-04-29 10:00
PULocationID                    237
NoOfPickups                       4
Name: 52, dtype: object


In [24]:
# PERSONAL NOTES:
# sort by datetime after compiling unique csv
# interpolate missing values every hour